In [ ]:
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict

In [ ]:
df = pd.read_feather('data/feather/FRS_simulations_with_sex_and_lung_cancer.feather')

In [ ]:
ds = Dataset.from_pandas(df)

In [ ]:
complex_cols = list(ds.features.keys())[:-2]

def transform(complex_sample):
    data = []
    for colname in complex_cols:
        val = complex(complex_sample[colname])
        data.append([val.real, val.imag, np.abs(val), np.angle(val)])
    return {'data': data}

ds = ds.map(transform, remove_columns=complex_cols)
ds = ds.rename_columns({'Label': 'label', 'sex_numeric': 'sex_label'})
ds = ds.with_format('torch')

In [ ]:
split1 = ds.train_test_split(.2)
split2 = split1['test'].train_test_split(.5)
dsd = DatasetDict({
    'train': split1['train'],
    'dev': split2['train'],
    'test': split2['test']
})

In [ ]:
dsd

In [ ]:
dsd.save_to_disk('data/dataset')